# 1. Data Preprocessing

In [7]:
import pandas as pd
import numpy as np

data = pd.read_csv('archive/EEG_data.csv')
data.shape

(12811, 15)

In [8]:
data.head()

,SubjectID,VideoID,Attention,Mediation,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,predefinedlabel,user-definedlabeln
0,0.0,0.0,56.0,43.0,278.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0.0,0.0
1,0.0,0.0,40.0,35.0,-50.0,73787.0,28083.0,1439.0,2240.0,2746.0,3687.0,5293.0,2740.0,0.0,0.0
2,0.0,0.0,47.0,48.0,101.0,758353.0,383745.0,201999.0,62107.0,36293.0,130536.0,57243.0,25354.0,0.0,0.0
3,0.0,0.0,47.0,57.0,-5.0,2012240.0,129350.0,61236.0,17084.0,11488.0,62462.0,49960.0,33932.0,0.0,0.0
4,0.0,0.0,44.0,53.0,-8.0,1005145.0,354328.0,37102.0,88881.0,45307.0,99603.0,44790.0,29749.0,0.0,0.0


In [11]:
# bandpass filter
from scipy.signal import butter, filtfilt

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

lowcut = 0.1
highcut = 30.0
fs = 100.0  # Sampling frequency

filtered_data = bandpass_filter(data['Raw'], lowcut, highcut, fs)
print(filtered_data)

[ -23.27277976 -262.63099206 -294.50212759 ... -324.52822167  -68.5035079
   -1.49463393]


confusion threshold? literature
1 vs 7
12 vs 67
spectrum/*csp*
pa3 csp code